<div class="alert alert-block alert-success">
<h1><b>PROJET 8 - </b>Participez à la conception d'une voiture autonome</h1>
<h2>Notebook Scripts</h2></div>

<img src="future_vision_transport.png" width="250" height="250">

## Sommaire<hr>
[INTRODUCTION & PRESENTATION](#section_0)<br>
[A - IMPORTATIONS, DATA COLLECTION & INITIAL DATAFRAME](#section_0A)<br>
[B - EXPLORATORY DATA ANALYSIS](#section_0B)<br>
[C - TEXT PREPROCESSING](#section_0C)<br>
[D - WORD EMBEDDINGS](#section_0D)<br>
&emsp;[1 - Our own Word2Vec](#section_0D1)<br>
&emsp;[2 - Google's Word2Vec](#section_0D2)<br>
&emsp;[3 - GloVe word embeddings](#section_0D3)<br>
&emsp;[4 - FastText](#section_0D4)<br>
[E - DATA AND DATASET PREPARATION FOR ALGORITHMS](#section_0E)<br>
[F - PERFORMANCE METRICS DEFINITION](#section_0F)<br><hr>

<h2><div class="alert alert-block alert-success" id="section_0">
INTRODUCTION & PRESENTATION</div></h2>

<h3><div class="alert alert-block alert-info" id="section_0A">
<b>A - </b>IMPORTATIONS, DATA COLLECTION & INITIAL DATAFRAME</div></h3>

In [ ]:
%matplotlib inline

# Importation of Python modules and methods.
import warnings

# Importation of Machine Learning libraries.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle5 as pickle
import seaborn as sns

# Importation of scikit-learn functions or classes.
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, 
    classification_report,
    ConfusionMatrixDisplay,
    confusion_matrix
)

# Importation of TensorFlow/Keras functions or classes.
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import (
    Bidirectional,
    Dense,
    Dropout,
    Embedding,
    LSTM,
)
from tensorflow.keras.models import Sequential, load_model